In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix

# Path to image folders, 
data_path = fr'../Datasets/full/Training'
#data_path = fr'../Datasets/config'

def remove_white_background(pixels):
    newPixels = []
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0
        newPixels.append(pixel)
    
    return newPixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]


def blueify(pixels):
    return [b for r, g, b in pixels]


def get_rgb_pixels_onehot_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y


def process_files(src):
    X_red_train = []
    X_red_validation = []
    X_red_test = []
    X_green_train = []
    X_green_validation = []
    X_green_test = []
    X_blue_train = []
    X_blue_validation = []
    X_blue_test = []
    all_pixels, y = get_rgb_pixels_onehot_labels(src)

    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
    
    for pixels in X_train:       
        X_red_train.append(redify(pixels.copy()))
        X_green_train.append(greenify(pixels.copy()))
        X_blue_train.append(blueify(pixels.copy()))
        
    for pixels in X_validation:       
        X_red_validation.append(redify(pixels.copy()))
        X_green_validation.append(greenify(pixels.copy()))
        X_blue_validation.append(blueify(pixels.copy()))
        
    for pixels in X_test:       
        X_red_test.append(redify(pixels.copy()))
        X_green_test.append(greenify(pixels.copy()))
        X_blue_test.append(blueify(pixels.copy()))
    
    
    print("Finished \n")
    return np.asarray(X_red_train), np.asarray(X_red_validation), np.asarray(X_red_test), np.asarray(X_green_train), np.asarray(X_green_validation), np.asarray(X_green_test), np.asarray(X_blue_train), np.asarray(X_blue_validation), np.asarray(X_blue_test), y_train, y_validation, y_test


def get_youdens_index(predictions, Y):
    # Calculate true positive/negative and false positive/negative
    tp = sum((Y == predictions) * (Y == 1) * 1)
    tn = sum((Y == predictions) * (Y == 0) * 1)
    fp = sum((Y != predictions) * (Y == 0) * 1)
    fn = sum((Y != predictions) * (Y == 1) * 1)
    
    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    
    result = sensitivity - (1 - specificity)
    # Put it in a dateframe for nicer visuals
    df = pd.DataFrame({'Youdens Index': result})
    pd.set_option('display.max_rows', 200)
    
    return df

In [2]:
# Call process_files and assign variables
X_red_train, X_red_validation, X_red_test, X_green_train, X_green_validation, X_green_test, X_blue_train, X_blue_validation, X_blue_test, Y_train, Y_validation, Y_test = process_files(data_path)

Starting...
Finished 



In [3]:
import time
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# Red Config HL Test
print("Red Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, random_state=1, max_iter=500)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")

# Green Config HL Test
print("Green Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, random_state=1, max_iter=500)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
# Blue Config HL Test
print("Blue Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, random_state=1, max_iter=500)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")

Red Config HL Tests
Hidden layers:  432
Score:  0.9287703274671419 Time:  175.45528483390808
     Youdens Index
0         0.969353
1         0.999720
2         0.991402
3         0.992254
4         0.992136
5         0.969129
6         0.967213
7         0.961383
8         0.870229
9         0.991192
10        0.984116
11        0.998710
12        0.971629
13        0.884991
14        0.990948
15        0.969297
16        0.992198
17        0.933884
18        0.961720
19        0.949832
20        1.000000
21        0.954198
22        0.717557
23        0.808936
24        0.961552
25        0.987279
26        0.999888
27        0.883023
28        0.964185
29        1.000000
30        1.000000
31        0.961832
32        0.884241
33        1.000000
34        0.816233
35        0.983053
36        0.991646
37        0.933333
38        0.999776
39        0.954030
40        0.943888
41        0.983957
42        0.923939
43        0.999944
44        0.996198
45        0.854906
46        0.99

Score:  0.8555914457562932 Time:  119.50080132484436
     Youdens Index
0         0.969297
1         0.971931
2         0.842236
3         0.823642
4         0.961072
5         0.991637
6         0.942118
7         0.952235
8         0.748092
9         0.921459
10        0.938707
11        0.923552
12        0.964886
13        0.785362
14        0.982176
15        0.931186
16        0.969353
17        0.941812
18        0.938875
19        0.799664
20        0.991814
21        0.953525
22        0.900483
23        0.997195
24        0.976538
25        0.881115
26        0.992254
27        0.922338
28        0.943656
29        1.000000
30        0.999776
31        0.930793
32        0.834543
33        0.999551
34        0.876348
35        0.899103
36        0.886067
37        0.914286
38        0.790822
39        0.930512
40        0.967888
41        0.973093
42        0.922764
43        0.954142
44        1.000000
45        0.967334
46        0.969129
47        0.999495
48        0.9693

Score:  0.7464357317888171 Time:  156.56884837150574
     Youdens Index
0         0.991974
1         0.974622
2         0.101450
3         0.693927
4         0.882319
5         0.997924
6         0.843702
7         0.755276
8         0.906153
9         0.686788
10        0.900483
11        0.907836
12        0.903763
13        0.807870
14        0.929320
15        0.678997
16        0.808711
17        0.916514
18        0.869668
19        0.557436
20        0.967199
21        0.679109
22        0.526605
23        0.936687
24        0.860968
25        0.705488
26        0.984733
27        0.923351
28        0.810606
29        0.999944
30        0.984733
31        0.992142
32        0.718672
33        0.786260
34        0.647509
35        0.807997
36        0.691057
37        0.799832
38        0.823327
39        0.824035
40        0.911551
41        0.866141
42        0.707295
43        0.679221
44        0.992339
45        0.747755
46        0.770319
47        0.541143
48        0.9307

Score:  0.9457006014702606 Time:  110.86603355407715
     Youdens Index
0         0.992366
1         1.000000
2         0.913726
3         0.975641
4         0.984272
5         0.999832
6         0.991635
7         0.969410
8         0.984340
9         0.982609
10        0.893130
11        0.961495
12        0.977528
13        0.992366
14        0.964912
15        0.961552
16        0.977099
17        0.925620
18        0.984452
19        0.874944
20        0.975610
21        0.931073
22        0.946397
23        0.992030
24        1.000000
25        0.973093
26        0.999944
27        0.984659
28        0.948901
29        1.000000
30        1.000000
31        0.999944
32        0.942037
33        0.992254
34        0.839134
35        0.924439
36        0.942473
37        0.971373
38        0.992000
39        0.915862
40        0.943888
41        0.940895
42        0.898734
43        0.946565
44        0.977186
45        0.961327
46        0.946284
47        0.999607
48        0.9691

In [4]:
# default alpha=0.0001

print("Red Config Alpha Tests")
# Red Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=(216, 108), alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")
    
print("Green Config Alpha Tests")
# Green Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=(216, 108), alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
print("Blue Config Alpha Tests")
# Blue Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=(216, 108), alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")    

Red Config Alpha Tests
Alpha:  0.0001
Score:  0.9056582757852528 Time:  117.20110535621643
     Youdens Index
0         0.984565
1         0.949468
2         0.976002
3         0.908116
4         0.829401
5         0.998990
6         0.901303
7         0.968512
8         0.892625
9         0.991024
10        0.862147
11        0.991974
12        0.949382
13        0.534071
14        0.736842
15        0.954198
16        0.976707
17        0.942149
18        0.977043
19        0.883165
20        0.991870
21        0.893130
22        0.999776
23        0.984172
24        0.961832
25        0.978328
26        0.992366
27        0.928315
28        0.878173
29        1.000000
30        0.999944
31        0.999776
32        0.818126
33        0.999551
34        0.884823
35        0.916162
36        0.918699
37        0.923810
38        0.935944
39        0.946509
40        0.968000
41        0.989249
42        0.949087
43        0.961664
44        1.000000
45        0.938034
46        0.9465

Score:  0.9470372020494542 Time:  94.18238830566406
     Youdens Index
0         0.999944
1         1.000000
2         0.999159
3         0.969353
4         0.891361
5         0.992030
6         1.000000
7         0.977043
8         0.854962
9         0.947826
10        0.961720
11        0.992310
12        0.898764
13        0.991918
14        0.991172
15        0.954198
16        0.984565
17        0.958622
18        0.954142
19        0.949664
20        0.999832
21        0.999776
22        1.000000
23        0.954198
24        0.969297
25        0.978497
26        1.000000
27        0.989228
28        0.984603
29        1.000000
30        0.992366
31        0.999776
32        0.975151
33        0.999439
34        0.847216
35        0.958333
36        0.958789
37        0.990308
38        0.976000
39        0.854962
40        0.951944
41        0.994652
42        0.886020
43        0.992254
44        0.988593
45        0.984228
46        0.908397
47        0.998822
48        0.99236

Score:  0.9385720650478948 Time:  85.60066509246826
     Youdens Index
0         1.000000
1         0.999888
2         0.937500
3         0.954030
4         0.991631
5         0.991918
6         0.999944
7         0.999046
8         0.976482
9         0.998991
10        0.999719
11        0.969466
12        0.971741
13        0.961720
14        0.745614
15        0.968961
16        0.992030
17        0.966942
18        0.938819
19        0.924720
20        0.934959
21        0.969297
22        0.915918
23        0.992310
24        0.961383
25        0.994146
26        1.000000
27        0.999944
28        0.997860
29        1.000000
30        1.000000
31        0.931298
32        0.809861
33        0.991918
34        0.938595
35        0.974608
36        0.918643
37        0.961849
38        0.991832
39        0.953301
40        0.959888
41        0.989080
42        0.923995
43        0.976538
44        0.992339
45        0.999944
46        0.961776
47        0.992366
48        0.99988

Score:  0.9405212742258855 Time:  89.85862016677856
     Youdens Index
0         0.984733
1         0.957927
2         0.905970
3         0.877694
4         0.945680
5         0.984677
6         0.983382
7         0.992086
8         0.961776
9         0.982441
10        0.946565
11        0.976819
12        0.971910
13        0.992198
14        0.990892
15        0.946284
16        0.969241
17        0.975151
18        0.953974
19        0.941274
20        0.943089
21        0.953750
22        0.969185
23        0.999776
24        0.992198
25        0.999719
26        0.999663
27        0.989679
28        0.938411
29        0.999888
30        1.000000
31        0.999944
32        0.801597
33        0.992310
34        0.869724
35        0.949551
36        0.975554
37        0.923697
38        0.944000
39        0.900763
40        0.975888
41        0.946412
42        0.911225
43        0.976482
44        0.973328
45        0.946004
46        0.732824
47        0.999271
48        0.98467

In [5]:
# Red
mlpc_red =  MLPClassifier(hidden_layer_sizes=((216, 108)), alpha=0.1, activation='relu', solver='adam', random_state=1, max_iter=500)
mlpc_red.fit(X_red_train, Y_train)
vector_red = mlpc_red.predict(X_red_train)

# Green
mlpc_green = MLPClassifier(hidden_layer_sizes=((216, 108)), alpha=0.1, activation='relu', solver='adam', random_state=1, max_iter=500)
mlpc_green.fit(X_green_train, Y_train)
vector_green = mlpc_green.predict(X_green_train)

# Blue
mlpc_blue = MLPClassifier(hidden_layer_sizes=((216, 108)), alpha=0.1, activation='relu', solver='adam', random_state=1, max_iter=500)
mlpc_blue.fit(X_blue_train, Y_train)
vector_blue = mlpc_blue.predict(X_blue_train)

# Validation
vector_red_val = mlpc_red.predict(X_red_validation)
vector_green_val = mlpc_green.predict(X_green_validation)
vector_blue_val = mlpc_blue.predict(X_blue_validation)


print("Red Model Validation")
print(classification_report(Y_validation, vector_red_val, zero_division=0))
print(get_youdens_index(vector_red_val, Y_validation), "\n")

print("Green Model Validation")
print(classification_report(Y_validation, vector_green_val, zero_division=0))
print(get_youdens_index(vector_green_val, Y_validation), "\n")

print("Blue Model Validation")
print(classification_report(Y_validation, vector_blue_val, zero_division=0))
print(get_youdens_index(vector_blue_val, Y_validation), "\n")

Red Model Validation
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       131
           1       1.00      0.94      0.97       119
           2       0.96      0.91      0.94       128
           3       0.83      0.97      0.89       131
           4       0.98      0.95      0.96       129
           5       0.99      0.82      0.90       131
           6       0.98      1.00      0.99       122
           7       0.95      0.85      0.90       131
           8       0.90      0.79      0.84       131
           9       0.98      0.92      0.95       115
          10       0.97      0.98      0.97       131
          11       0.83      1.00      0.91       131
          12       0.94      0.98      0.96       178
          13       0.92      0.82      0.87       131
          14       1.00      0.97      0.99       114
          15       0.98      1.00      0.99       131
          16       0.94      0.99      0.96       131
      

     Youdens Index
0         1.000000
1         0.999888
2         0.937500
3         0.954030
4         0.991631
5         0.991918
6         0.999944
7         0.999046
8         0.976482
9         0.998991
10        0.999719
11        0.969466
12        0.971741
13        0.961720
14        0.745614
15        0.968961
16        0.992030
17        0.966942
18        0.938819
19        0.924720
20        0.934959
21        0.969297
22        0.915918
23        0.992310
24        0.961383
25        0.994146
26        1.000000
27        0.999944
28        0.997860
29        1.000000
30        1.000000
31        0.931298
32        0.809861
33        0.991918
34        0.938595
35        0.974608
36        0.918643
37        0.961849
38        0.991832
39        0.953301
40        0.959888
41        0.989080
42        0.923995
43        0.976538
44        0.992339
45        0.999944
46        0.961776
47        0.992366
48        0.999888
49        0.928571
50        0.892961
51        0.

In [6]:
# Stack the rgb predictions to get combi model values 
X_combined_train = np.column_stack((vector_red, vector_green, vector_blue)) # shape (1745, 18) - 18 features
X_combined_val = np.column_stack((vector_red_val, vector_green_val, vector_blue_val)) # shape (582, 18) - 18 features

# Combi Config HL Test
print("Combi Config HL Tests")
for hl in [(15), (8), (8, 4)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")


print("Combi Config Alpha Tests")
# Combi Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=15, alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")

Combi Config HL Tests
Hidden layers:  15
Score:  0.9857429271552685 Time:  24.771114349365234
     Youdens Index
0         1.000000
1         1.000000
2         0.992131
3         0.977099
4         0.992192
5         0.977099
6         0.991803
7         0.992310
8         0.992310
9         1.000000
10        1.000000
11        1.000000
12        0.994382
13        0.984733
14        0.991228
15        0.984733
16        0.961832
17        0.950413
18        0.977099
19        0.950000
20        0.991870
21        0.969466
22        1.000000
23        1.000000
24        1.000000
25        0.989305
26        0.992366
27        0.984772
28        1.000000
29        1.000000
30        1.000000
31        1.000000
32        0.950413
33        0.992366
34        0.954198
35        0.983333
36        0.959350
37        0.961905
38        0.968000
39        0.954198
40        0.984000
41        1.000000
42        0.962025
43        0.992366
44        1.000000
45        0.992366
46        0.9

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Score:  0.9074961015816441 Time:  195.7900652885437
     Youdens Index
0         0.992198
1         0.983193
2         0.968357
3         0.984116
4         0.968656
5         0.953918
6         0.966989
7         0.953694
8         0.930400
9         0.947434
10        0.984452
11        0.946284
12        0.937640
13        0.007577
14        0.973628
15        0.938707
16        0.892569
17        0.909035
18        0.877638
19        0.916442
20        0.926773
21        0.969129
22        0.938707
23        0.953469
24        0.930681
25        0.973037
26        0.969466
27        0.968867
28        0.984321
29        0.999888
30        0.984677
31        0.992086
32        0.000000
33        0.984340
34        0.862091
35        0.924664
36        0.910345
37       -0.000840
38        0.968000
39        0.907836
40        0.951720
41        0.978441
42        0.911281
43        0.961552
44        0.988480
45        0.877750
46        0.946228
47        0.991974
48        0.96901

Score:  0.9340610380931165 Time:  81.5068187713623
     Youdens Index
0         0.969466
1         0.983193
2         0.929688
3         0.931298
4         0.937872
5         0.961832
6         0.983607
7         0.946565
8         0.954198
9         1.000000
10        0.946565
11        0.954198
12        0.932584
13        0.954198
14        0.964912
15        0.908397
16        0.908397
17        0.884298
18        0.893130
19        0.883333
20        0.934959
21        0.931298
22        0.893130
23        0.961832
24        0.946565
25        0.973262
26        1.000000
27        0.954315
28        0.989791
29        1.000000
30        0.984733
31        0.931298
32        0.743802
33        0.977099
34        0.893130
35        0.858333
36        0.886179
37        0.904762
38        0.960000
39        0.854962
40        0.912000
41        0.978610
42        0.784810
43        0.900763
44        0.992395
45        0.977099
46        0.915974
47        0.992366
48        0.908397

In [7]:
# Combi Train
mlpc_combi =  MLPClassifier(hidden_layer_sizes=(15), alpha=0.01, activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_combi.fit(X_combined_train, Y_train)

# Validation
print("Combined Model Validation")
combi_val = mlpc_combi.predict(X_combined_val)
print(classification_report(Y_validation, combi_val, zero_division=0))
print(get_youdens_index(combi_val, Y_validation), "\n")

Combined Model Validation
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131
           1       1.00      1.00      1.00       119
           2       1.00      0.98      0.99       128
           3       1.00      0.98      0.99       131
           4       0.99      0.98      0.99       129
           5       1.00      0.97      0.98       131
           6       1.00      0.98      0.99       122
           7       1.00      0.99      1.00       131
           8       1.00      0.98      0.99       131
           9       1.00      1.00      1.00       115
          10       1.00      0.99      1.00       131
          11       1.00      1.00      1.00       131
          12       1.00      0.97      0.99       178
          13       1.00      0.98      0.99       131
          14       1.00      0.98      0.99       114
          15       1.00      0.97      0.98       131
          16       1.00      0.99      1.00       131
 

In [8]:
vector_red_test = mlpc_red.predict(X_red_test)
vector_green_test = mlpc_green.predict(X_green_test)
vector_blue_test = mlpc_blue.predict(X_blue_test)

X_combined_test = np.column_stack((vector_red_test, vector_green_test, vector_blue_test)) # shape (582, 18) - 18 features

# Test Final
print("Combined Model Test")
combi_test = mlpc_combi.predict(X_combined_test)
print(classification_report(Y_test, combi_test, zero_division=0))
print(get_youdens_index(combi_test, Y_test), "\n")

Combined Model Test
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131
           1       1.00      0.98      0.99       118
           2       1.00      0.95      0.98       128
           3       1.00      0.98      0.99       131
           4       1.00      0.95      0.98       128
           5       1.00      0.97      0.98       132
           6       1.00      0.97      0.98       122
           7       1.00      0.98      0.99       131
           8       1.00      0.98      0.99       131
           9       1.00      0.97      0.98       115
          10       1.00      1.00      1.00       131
          11       1.00      1.00      1.00       131
          12       1.00      0.98      0.99       179
          13       1.00      0.98      0.99       131
          14       0.99      0.98      0.99       114
          15       1.00      0.97      0.98       132
          16       0.98      0.99      0.99       131
       